In [40]:
#import libraries
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import re

In [2]:
pd.options.display.max_columns = 100

In [3]:
#load data and view first few rows
df = pd.read_csv("fifa21 raw data v2.csv",dtype={'Hits':'object'})

df.head()

,ID,Name,LongName,photoUrl,playerUrl,Nationality,Age,↓OVA,POT,Club,Contract,Positions,Height,Weight,Preferred Foot,BOV,Best Position,Joined,Loan Date End,Value,Wage,Release Clause,Attacking,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Skill,Dribbling,Curve,FK Accuracy,Long Passing,Ball Control,Movement,Acceleration,Sprint Speed,Agility,Reactions,Balance,Power,Shot Power,Jumping,Stamina,Strength,Long Shots,Mentality,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Defending,Marking,Standing Tackle,Sliding Tackle,Goalkeeping,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,Total Stats,Base Stats,W/F,SM,A/W,D/W,IR,PAC,SHO,PAS,DRI,DEF,PHY,Hits
0,158023,L. Messi,Lionel Messi,https://cdn.sofifa.com/players/158/023/21_60.png,http://sofifa.com/player/158023/lionel-messi/2...,Argentina,33,93,93,\n\n\n\nFC Barcelona,2004 ~ 2021,"RW, ST, CF",170cm,72kg,Left,93,RW,"Jul 1, 2004",NaN,€103.5M,€560K,€138.4M,429,85,95,70,91,88,470,96,93,94,91,96,451,91,80,91,94,95,389,86,68,72,69,94,347,44,40,93,95,75,96,91,32,35,24,54,6,11,15,14,8,2231,466,4 ★,4★,Medium,Low,5 ★,85,92,91,95,38,65,771
1,20801,Cristiano Ronaldo,C. Ronaldo dos Santos Aveiro,https://cdn.sofifa.com/players/020/801/21_60.png,http://sofifa.com/player/20801/c-ronaldo-dos-s...,Portugal,35,92,92,\n\n\n\nJuventus,2018 ~ 2022,"ST, LW",187cm,83kg,Right,92,ST,"Jul 10, 2018",NaN,€63M,€220K,€75.9M,437,84,95,90,82,86,414,88,81,76,77,92,431,87,91,87,95,71,444,94,95,84,78,93,353,63,29,95,82,84,95,84,28,32,24,58,7,11,15,14,11,2221,464,4 ★,5★,High,Low,5 ★,89,93,81,89,35,77,562
2,200389,J. Oblak,Jan Oblak,https://cdn.sofifa.com/players/200/389/21_60.png,http://sofifa.com/player/200389/jan-oblak/210006/,Slovenia,27,91,93,\n\n\n\nAtlético Madrid,2014 ~ 2023,GK,188cm,87kg,Right,91,GK,"Jul 16, 2014",NaN,€120M,€125K,€159.4M,95,13,11,15,43,13,109,12,13,14,40,30,307,43,60,67,88,49,268,59,78,41,78,12,140,34,19,11,65,11,68,57,27,12,18,437,87,92,78,90,90,1413,489,3 ★,1★,Medium,Medium,3 ★,87,92,78,90,52,90,150
3,192985,K. De Bruyne,Kevin De Bruyne,https://cdn.sofifa.com/players/192/985/21_60.png,http://sofifa.com/player/192985/kevin-de-bruyn...,Belgium,29,91,91,\n\n\n\nManchester City,2015 ~ 2023,"CAM, CM",181cm,70kg,Right,91,CAM,"Aug 30, 2015",NaN,€129M,€370K,€161M,407,94,82,55,94,82,441,88,85,83,93,92,398,77,76,78,91,76,408,91,63,89,74,91,408,76,66,88,94,84,91,186,68,65,53,56,15,13,5,10,13,2304,485,5 ★,4★,High,High,4 ★,76,86,93,88,64,78,207
4,190871,Neymar Jr,Neymar da Silva Santos Jr.,https://cdn.sofifa.com/players/190/871/21_60.png,http://sofifa.com/player/190871/neymar-da-silv...,Brazil,28,91,91,\n\n\n\nParis Saint-Germain,2017 ~ 2022,"LW, CAM",175cm,68kg,Right,91,LW,"Aug 3, 2017",NaN,€132M,€270K,€166.5M,408,85,87,62,87,87,448,95,88,89,81,95,453,94,89,96,91,83,357,80,62,81,50,84,356,51,36,87,90,92,93,94,35,30,29,59,9,9,15,15,11,2175,451,5 ★,5★,High,Medium,5 ★,91,85,86,94,36,59,595


In [4]:
df.shape

(18979, 77)

## Exploratory Data Analysis

In [5]:
#create a copy for cleaning
df0 = df.copy()

In [6]:
df0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18979 entries, 0 to 18978
Data columns (total 77 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   ID                18979 non-null  int64 
 1   Name              18979 non-null  object
 2   LongName          18979 non-null  object
 3   photoUrl          18979 non-null  object
 4   playerUrl         18979 non-null  object
 5   Nationality       18979 non-null  object
 6   Age               18979 non-null  int64 
 7   ↓OVA              18979 non-null  int64 
 8   POT               18979 non-null  int64 
 9   Club              18979 non-null  object
 10  Contract          18979 non-null  object
 11  Positions         18979 non-null  object
 12  Height            18979 non-null  object
 13  Weight            18979 non-null  object
 14  Preferred Foot    18979 non-null  object
 15  BOV               18979 non-null  int64 
 16  Best Position     18979 non-null  object
 17  Joined      

In [7]:
df0.nunique()

ID           18979
Name         17920
LongName     18852
photoUrl     18979
playerUrl    18979
             ...  
PAS             68
DRI             69
DEF             78
PHY             63
Hits           439
Length: 77, dtype: int64

 ### Data Cleaning Issues
 
 1. Remove non-numeric symbols from the following columns:IR, W/F, SM, Weight, Height
 2. Expand Contract column into contract_start and contract_end columns and convert to datetime dtype
 8. Create new column: contract_status. It indicates if a player is Free, On Loan or Contracted, Get values to fill from the contract_start column
 9. Move values in contract_start column that are loan Start dates into a new column: loan_start
 11. Replace Free and loan_dates in contract_start with Nan and replace None in contract_end column with Nan
 3. Convert Joined and Loan Date End to datetime dtype
 4. Expand Position Column to each available position and Indicate a players compatibility with that position with True/False
 5. Remove the newline characters in the Club column
 6. Remove the arrow down symbol in the OVA column name
 7. Remove the currency symbol from the Value,Wage and Release Clause columns.Also,remove the currency abbreviation then multiply the remaining number by its corresponding value, such as 1000 for 'K' or 1000000 for 'M', 
 11. Replace NaN values in the Hits columns with zero, convert Hits to int dtype. Adjust values with K e.g: 1.6K, multiply them by 1000


#### 1. Remove non-numeric symbols from the following columns:IR, W/F, SM, Weight, Height


In [8]:
# Remove non-numeric symbols from the following columns:IR, W/F, SM, Weight, Height
star_cols = ['W/F', 'SM', 'IR','Weight','Height']
for col in star_cols:
    # remove star
    df0[f"{col}"] = df0[f"{col}"].str.replace(pat=r"\D",repl="",regex=True)
    #convert them to int dtype
    df0[f"{col}"] = df0[f"{col}"].astype('int')

#### 2. Expand Contract column into contract_start and contract_end columns and convert to datetime dtype

In [9]:
#Create a new DataFrame by splitting and expanding the Contract Column
contract_expanded = df0.Contract.str.split("~",expand=True)

#rename the columns of contract_expanded i.e 0=contract_start, 1=contract_end
contract_expanded.columns = ('contract_start','contract_end')

#filter on rows that have non numeric values
contract_expanded[contract_expanded.contract_start.str.contains(r'[a-zA-Z]+') | contract_expanded.contract_end.str.contains(r'[a-zA-Z]+')]


,contract_start,contract_end
205,"Jun 30, 2021 On Loan",None
248,"Jun 30, 2021 On Loan",None
254,"Jun 30, 2021 On Loan",None
289,Free,None
292,Free,None
...,...,...
18472,"Aug 31, 2021 On Loan",None
18571,"Jun 30, 2021 On Loan",None
18600,"Dec 31, 2020 On Loan",None
18622,"Dec 31, 2020 On Loan",None


#### 3. Create new column: contract_status. It indicates if a player is Free, On Loan or Contracted, Get values to fill from the contract_start column

In [10]:
# Create new column: contract_status. It indicates if a player is Free, On Loan or Contracted, Get values to fill from the contract_start column
contract_expanded['contract_status'] = contract_expanded.contract_start.apply(lambda x: "Contracted" if x.strip(" ").isnumeric() else x[-4:])
contract_expanded.head()

,contract_start,contract_end,contract_status
0,2004,2021,Contracted
1,2018,2022,Contracted
2,2014,2023,Contracted
3,2015,2023,Contracted
4,2017,2022,Contracted


#### 4. Move values in contract_start column that are loan Start dates into a new column: loan_start

In [11]:
# define function to extract date from contract_start column
def extract_date(string:str) -> str:
    """This function extracts dates in the "MMM DD, YYYY" format (e.g. "Jun 30, 2021") from a string
    Parameters:
    -----------
    string : str
        The input string to search for dates.
        
    Returns:
    --------
    str or None
        The matched date in the "MMM DD, YYYY" format, or None if no match is found.
    """
    match = re.search(r'\b\w{3}\s\d{1,2},\s\d{4}\b', string)
    if match:
        return match.group(0)
    else:
        return None


In [12]:
# Move values in contract_start column that are loan Start dates into a new column: loan_start
contract_expanded['loan_start'] = contract_expanded.contract_start.apply(extract_date)
contract_expanded

,contract_start,contract_end,contract_status,loan_start
0,2004,2021,Contracted,None
1,2018,2022,Contracted,None
2,2014,2023,Contracted,None
3,2015,2023,Contracted,None
4,2017,2022,Contracted,None
...,...,...,...,...
18974,2018,2022,Contracted,None
18975,2020,2021,Contracted,None
18976,2019,2020,Contracted,None
18977,2020,2022,Contracted,None


#### 5. Replace Free and loan_dates in contract_start with Nan and replace None in contract_end column with Nan

In [13]:
# convert to datetime and replace errors with Nan i.e values that can be converted to numeric
contract_expanded['contract_start'] = pd.to_datetime(contract_expanded.contract_start,errors='coerce')
contract_expanded['contract_end'] = pd.to_datetime(contract_expanded.contract_end,errors='coerce')
contract_expanded

,contract_start,contract_end,contract_status,loan_start
0,2004-01-01,2021-01-01,Contracted,None
1,2018-01-01,2022-01-01,Contracted,None
2,2014-01-01,2023-01-01,Contracted,None
3,2015-01-01,2023-01-01,Contracted,None
4,2017-01-01,2022-01-01,Contracted,None
...,...,...,...,...
18974,2018-01-01,2022-01-01,Contracted,None
18975,2020-01-01,2021-01-01,Contracted,None
18976,2019-01-01,2020-01-01,Contracted,None
18977,2020-01-01,2022-01-01,Contracted,None


In [14]:
#join the contract expanded dataframe with the main(df0) dataframe
df02 = df0.join(contract_expanded)
#drop Contract column
df02.drop('Contract',inplace=True,axis=1)

#### 6. Convert Joined and Loan Date End to datetime dtype

In [15]:
# Convert Joined and Loan Date End to datetime dtype
df02['Joined'] = pd.to_datetime(df02.Joined)
df02['Loan Date End'] = pd.to_datetime(df02['Loan Date End'] )

#### 7. Expand Position Column to each available position and Indicate a players compatibility with that position with True/False

In [16]:
# join all values in Position Column into one string.
positions = ",".join(df02.Positions.unique())
#split the string with the ',' delimeter
positions = [x.strip(" ") for x in positions.split(',') ]
#find unique values by putting it in a set dtype
unique_positions = [x.strip(" ") for x in set(positions) ]
unique_positions.sort()

In [17]:
pos_df = df02.Positions.to_frame()
# check if Positions column contains the position and pass True if true and otherwise False
for pos in unique_positions:
    pos_df[f"{pos}"] = pos_df.Positions.str.contains(pos)
pos_df

,Positions,CAM,CB,CDM,CF,CM,GK,LB,LM,LW,LWB,RB,RM,RW,RWB,ST
0,"RW, ST, CF",False,False,False,True,False,False,False,False,False,False,False,False,True,False,True
1,"ST, LW",False,False,False,False,False,False,False,False,True,False,False,False,False,False,True
2,GK,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False
3,"CAM, CM",True,False,False,False,True,False,False,False,False,False,False,False,False,False,False
4,"LW, CAM",True,False,False,False,False,False,False,False,True,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18974,CB,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False
18975,CM,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False
18976,CM,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False
18977,RW,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False


In [18]:
#join expanded positions columns to df02 
df03 = df02.join(pos_df.iloc[:,1:])

#### 8. Remove the newline characters in the Club column

In [19]:
df03['Club'] = df03.Club.str.replace('\n', '', regex=True).str.lstrip(" ").str.rstrip(" ")

#### 9. Remove the arrow down symbol in the OVA column name

In [20]:
df03.rename(columns={'↓OVA':'OVA'},inplace=True)

#### 10. Remove the currency symbol from the Value,Wage and Release Clause columns.Also,remove the currency abbreviation then multiply the remaining number by its corresponding value, such as 1000 for 'K' or 1000000 for 'M',

In [26]:
def clean_currency(x):
    """
    This function takes a string parameter and removes any trailing whitespace
    character. If the string starts with '€' and ends with 'M' or 'K', the 
    function converts the numeric value to million or thousand respectively, 
    and returns an integer. If the string starts with '€' but does not end with 
    'M' or 'K', it removes the '€' sign and returns the remaining string. 
    If the string does not start with '€', the function returns the original
    string.

    Args:
        x: A string containing a currency value.

    Returns:
        An float or string depending on the input.

    Raises:
        None.
    """
    x = x.rstrip(" ")
    if x[-1] == "M" and x[0] == "€":
        return float(x[1:-1]) * 1_000_000
    elif x[-1] == "K" and x[0] == "€":
        return float(x[1:-1]) * 1_000
    else:
        if x[0] == "€":
            return x[1:]
        else:
            return x

#Creates a vectorized version of the clean_currency function that can be applied to an array or list of values.
clean_currency_v2 = np.vectorize(clean_currency) 

In [ ]:
df03[['Wage','Value','Release Clause']] = df03[['Wage','Value','Release Clause']].apply(clean_currency_v2)

#### 11. Replace NaN values in the Hits columns with zero, convert Hits to int dtype. Adjust values with K e.g: 1.6K, multiply them by 1000

In [34]:
def clean_Hits(x):
    """
    Cleans a string representing a number of hits by removing any trailing spaces 
    and scaling the number appropriately if it is followed by 'K'.

    Args:
        x (str): A string representing a number of hits, with or without 'K' 
        suffix to denote thousands.

    Returns:
        float or str: The input value as a float if it represents a number of hits
        in thousands (i.e., ends with 'K'), scaled appropriately. Otherwise, returns 
        the input value as a string with trailing spaces removed.

    Raises:
        None

    Examples:
        >>> clean_Hits('10K ')
        10000.0
        >>> clean_Hits('500')
        '500'
    """
    if x.rstrip(" ")[-1] == "K":
        return  float(x.rstrip(" ")[0:-1]) * 1000
    else:
        return x

In [35]:
# fill missing values with 0 string
df03.Hits.fillna("0",inplace=True)

In [36]:
# Clean any non-numeric values in the 'Hits' column of the dataframe using the 'clean_Hits' function. 
# Specifically, use the 'contains' method to identify rows where the 'Hits' column does not match the 
# regular expression for a decimal number with an optional decimal point. Apply the 'clean_Hits' function 
# to those rows to convert any 'K'-suffixed values to their scaled equivalent in thousands, and replace 
# the original values in the dataframe with the cleaned values.

df03.loc[~(df03.Hits.str.contains(r'^\d+(\.\d+)?$')),['Hits']] =  df03[~(df03.Hits.str.contains(r'^\d+(\.\d+)?$'))]['Hits'].apply(lambda x: clean_Hits(x))

/tmp/ipykernel_18460/686148438.py:7: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df03.loc[~(df03.Hits.str.contains(r'^\d+(\.\d+)?$')),['Hits']] =  df03[~(df03.Hits.str.contains(r'^\d+(\.\d+)?$'))]['Hits'].apply(lambda x: clean_Hits(x))


In [37]:
df03['Hits'] = df03.Hits.astype('int')

In [42]:
df03

,ID,Name,LongName,photoUrl,playerUrl,Nationality,Age,OVA,POT,Club,Positions,Height,Weight,Preferred Foot,BOV,Best Position,Joined,Loan Date End,Value,Wage,Release Clause,Attacking,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Skill,Dribbling,Curve,FK Accuracy,Long Passing,Ball Control,Movement,Acceleration,Sprint Speed,Agility,Reactions,Balance,Power,Shot Power,Jumping,Stamina,Strength,Long Shots,Mentality,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Defending,Marking,Standing Tackle,Sliding Tackle,Goalkeeping,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,Total Stats,Base Stats,W/F,SM,A/W,D/W,IR,PAC,SHO,PAS,DRI,DEF,PHY,Hits,contract_start,contract_end,contract_status,loan_start,CAM,CB,CDM,CF,CM,GK,LB,LM,LW,LWB,RB,RM,RW,RWB,ST
0,158023,L. Messi,Lionel Messi,https://cdn.sofifa.com/players/158/023/21_60.png,http://sofifa.com/player/158023/lionel-messi/2...,Argentina,33,93,93,FC Barcelona,"RW, ST, CF",170,72,Left,93,RW,2004-07-01,NaT,103500000.0,560000.0,138400000.0,429,85,95,70,91,88,470,96,93,94,91,96,451,91,80,91,94,95,389,86,68,72,69,94,347,44,40,93,95,75,96,91,32,35,24,54,6,11,15,14,8,2231,466,4,4,Medium,Low,5,85,92,91,95,38,65,771,2004-01-01,2021-01-01,Contracted,None,False,False,False,True,False,False,False,False,False,False,False,False,True,False,True
1,20801,Cristiano Ronaldo,C. Ronaldo dos Santos Aveiro,https://cdn.sofifa.com/players/020/801/21_60.png,http://sofifa.com/player/20801/c-ronaldo-dos-s...,Portugal,35,92,92,Juventus,"ST, LW",187,83,Right,92,ST,2018-07-10,NaT,63000000.0,220000.0,75900000.0,437,84,95,90,82,86,414,88,81,76,77,92,431,87,91,87,95,71,444,94,95,84,78,93,353,63,29,95,82,84,95,84,28,32,24,58,7,11,15,14,11,2221,464,4,5,High,Low,5,89,93,81,89,35,77,562,2018-01-01,2022-01-01,Contracted,None,False,False,False,False,False,False,False,False,True,False,False,False,False,False,True
2,200389,J. Oblak,Jan Oblak,https://cdn.sofifa.com/players/200/389/21_60.png,http://sofifa.com/player/200389/jan-oblak/210006/,Slovenia,27,91,93,Atlético Madrid,GK,188,87,Right,91,GK,2014-07-16,NaT,120000000.0,125000.0,159400000.0,95,13,11,15,43,13,109,12,13,14,40,30,307,43,60,67,88,49,268,59,78,41,78,12,140,34,19,11,65,11,68,57,27,12,18,437,87,92,78,90,90,1413,489,3,1,Medium,Medium,3,87,92,78,90,52,90,150,2014-01-01,2023-01-01,Contracted,None,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False
3,192985,K. De Bruyne,Kevin De Bruyne,https://cdn.sofifa.com/players/192/985/21_60.png,http://sofifa.com/player/192985/kevin-de-bruyn...,Belgium,29,91,91,Manchester City,"CAM, CM",181,70,Right,91,CAM,2015-08-30,NaT,129000000.0,370000.0,161000000.0,407,94,82,55,94,82,441,88,85,83,93,92,398,77,76,78,91,76,408,91,63,89,74,91,408,76,66,88,94,84,91,186,68,65,53,56,15,13,5,10,13,2304,485,5,4,High,High,4,76,86,93,88,64,78,207,2015-01-01,2023-01-01,Contracted,None,True,False,False,False,True,False,False,False,False,False,False,False,False,False,False
4,190871,Neymar Jr,Neymar da Silva Santos Jr.,https://cdn.sofifa.com/players/190/871/21_60.png,http://sofifa.com/player/190871/neymar-da-silv...,Brazil,28,91,91,Paris Saint-Germain,"LW, CAM",175,68,Right,91,LW,2017-08-03,NaT,132000000.0,270000.0,166500000.0,408,85,87,62,87,87,448,95,88,89,81,95,453,94,89,96,91,83,357,80,62,81,50,84,356,51,36,87,90,92,93,94,35,30,29,59,9,9,15,15,11,2175,451,5,5,High,Medium,5,91,85,86,94,36,59,595,2017-01-01,2022-01-01,Contracted,None,True,False,False,False,False,False,False,False,True,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18974,247223,Xia Ao,Ao Xia,https://cdn.sofifa.com/players/247/223/21_60.png,http://sofifa.com/pl

In [46]:
strikers = df03[df03['Best Position'].isin(["ST","CF"])]
sns.boxplot()

,ID,Name,LongName,photoUrl,playerUrl,Nationality,Age,OVA,POT,Club,Positions,Height,Weight,Preferred Foot,BOV,Best Position,Joined,Loan Date End,Value,Wage,Release Clause,Attacking,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Skill,Dribbling,Curve,FK Accuracy,Long Passing,Ball Control,Movement,Acceleration,Sprint Speed,Agility,Reactions,Balance,Power,Shot Power,Jumping,Stamina,Strength,Long Shots,Mentality,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Defending,Marking,Standing Tackle,Sliding Tackle,Goalkeeping,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,Total Stats,Base Stats,W/F,SM,A/W,D/W,IR,PAC,SHO,PAS,DRI,DEF,PHY,Hits,contract_start,contract_end,contract_status,loan_start,CAM,CB,CDM,CF,CM,GK,LB,LM,LW,LWB,RB,RM,RW,RWB,ST
1,20801,Cristiano Ronaldo,C. Ronaldo dos Santos Aveiro,https://cdn.sofifa.com/players/020/801/21_60.png,http://sofifa.com/player/20801/c-ronaldo-dos-s...,Portugal,35,92,92,Juventus,"ST, LW",187,83,Right,92,ST,2018-07-10,NaT,63000000.0,220000.0,75900000.0,437,84,95,90,82,86,414,88,81,76,77,92,431,87,91,87,95,71,444,94,95,84,78,93,353,63,29,95,82,84,95,84,28,32,24,58,7,11,15,14,11,2221,464,4,5,High,Low,5,89,93,81,89,35,77,562,2018-01-01,2022-01-01,Contracted,None,False,False,False,False,False,False,False,False,True,False,False,False,False,False,True
5,188545,R. Lewandowski,Robert Lewandowski,https://cdn.sofifa.com/players/188/545/21_60.png,http://sofifa.com/player/188545/robert-lewando...,Poland,31,91,91,FC Bayern München,ST,184,80,Right,91,ST,2014-07-01,NaT,111000000.0,240000.0,132000000.0,423,71,94,85,84,89,407,85,79,85,70,88,407,77,78,77,93,82,420,89,84,76,86,85,391,81,49,94,79,88,88,96,35,42,19,51,15,6,12,8,10,2195,457,4,4,High,Medium,4,78,91,78,85,43,82,248,2014-01-01,2023-01-01,Contracted,None,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True
8,231747,K. Mbappé,Kylian Mbappé,https://cdn.sofifa.com/players/231/747/21_60.png,http://sofifa.com/player/231747/kylian-mbappe/...,France,21,90,95,Paris Saint-Germain,"ST, LW, RW",178,73,Right,91,ST,2018-07-01,NaT,185500000.0,160000.0,203100000.0,408,78,91,73,83,83,394,92,79,63,70,90,458,96,96,92,92,82,404,86,77,86,76,79,341,62,38,91,80,70,84,100,34,34,32,42,13,5,7,11,6,2147,466,4,5,High,Low,3,96,86,78,91,39,76,1600,2018-01-01,2022-01-01,Contracted,None,False,False,False,False,False,False,False,False,True,False,False,False,True,False,True
15,165153,K. Benzema,Karim Benzema,https://cdn.sofifa.com/players/165/153/21_60.png,http://sofifa.com/player/165153/karim-benzema/...,France,32,89,89,Real Madrid,"CF, ST",185,81,Right,89,CF,2009-07-09,NaT,83500000.0,350000.0,108700000.0,426,75,88,91,86,86,404,87,81,73,73,90,388,77,72,79,91,69,400,84,79,78,80,79,363,63,39,90,87,84,90,89,47,24,18,41,13,11,5,5,7,2111,442,4,4,Medium,Low,4,74,85,81,86,40,76,216,2009-01-01,2022-01-01,Contracted,None,False,False,False,True,False,False,False,False,False,False,False,False,False,False,True
17,153079,S. Agüero,Sergio Agüero,https://cdn.sofifa.com/players/153/079/21_60.png,http://sofifa.com/player/153079/sergio-aguero/...,Argentina,32,89,89,Manchester City,ST,173,70,Right,89,ST,2011-07-28,NaT,83500000.0,300000.0,98100000.0,411,70,94,78,83,86,397,88,83,73,64,89,424,79,78,84,92,91,403,89,78,78,74,84,341,65,24,94,83,75,91,83,30,29,24,59,13,15,6,11,14,2118,439,4,4,High,Medium,4,78,90,77,88,33,73,154,2011-01-01,2021-01-01,Contracted,None,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18946,224866,J. Akintunde,James Akintunde,https://cdn.sofifa.com/players/224/866/21_60.png,http://sofifa.com/pla